# Pangeo REMA Example

Data live in Google Cloud Storage region `US-CENTRAL-1`.

Files have been encoded as COGs. The URLs have the format:

`https://storage.googleapis.com/pangeo-pgc/8m/{row}_{col}/{row}_{col}_8m_dem_COG_LZW.tif`

### Create Dask Cluster for Parallel Processing

In [ ]:
from dask.distributed import Client
from dask_kubernetes import KubeCluster

cluster = KubeCluster(n_workers=10)
client = Client(cluster)
cluster

In [ ]:
import numpy as np
import xarray as xr
import hvplot.xarray
from matplotlib import pyplot as plt
from rasterio import RasterioIOError
from tqdm.autonotebook import tqdm
%matplotlib inline

### Load a range of 8m images (lazily)

In [ ]:
uri_fmt = 'https://storage.googleapis.com/pangeo-pgc/8m/{i_idx:02d}_{j_idx:02d}/{i_idx:02d}_{j_idx:02d}_8m_dem_COG_LZW.tif'

chunksize = 8 * 512
rows = []
for i in tqdm(range(23, 16, -1)):
    cols = []
    for j in range(22, 34):
        uri = uri_fmt.format(i_idx=i, j_idx=j)
        try:
            dset = xr.open_rasterio(uri, chunks=chunksize)
            dset_masked = dset.where(dset > 0.0)
            cols.append(dset_masked)
            #print(uri)
        except RasterioIOError:
            pass
    rows.append(cols)

In [ ]:
[len(r) for r in rows]

### Concat into a single huge Xarray dataset

In [ ]:
dsets_rows = [xr.concat(row, 'x') for row in rows]
ds = xr.concat(dsets_rows, 'y', )
ds

### "Persist" -- load into cluster memory

In [ ]:
dsp = ds.persist()
data = dsp[0].data
data

### Calculate some averages over 100x100 pixel chunks

In [ ]:
elev_mean = dsp.coarsen(x=100, y=100).mean().load()
elev_std = dsp.coarsen(x=100, y=100).std().load()

In [ ]:
elev_mean.plot(figsize=(16, 6))
plt.title('Mean Eleveation');

In [ ]:
plt.title('Eleveation Standard Deviation');
np.log(elev_std).plot(figsize=(16, 6), vmax=2)